In [175]:
# Yasir Hassan
# Parameter Optimization
# Logistic Reg. to classify the Anticline images
#==================================================

# import libraries
from sklearn.linear_model import LogisticRegression
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import cv2
import scipy
import numpy as np
import os
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV 
from sklearn.linear_model import LogisticRegression 

In [176]:
# getData function definition
def get_data(folder):
    #Load the data and labels from the given folder
    X = []
    y = []
    
    for seismic_type in os.listdir(folder):
             if not seismic_type.startswith('.'):
                 if seismic_type in ['Class1']:
                     label = '0'
                 else:
                     label = '1'
                 for image_filename in os.listdir(folder + seismic_type):
                     img_file = cv2.imread(folder + seismic_type + '/' + image_filename, 0)
                     if img_file is not None:
                         # Downsample the image to 120, 160, 3
                         # img_file = scipy.misc.imresize(arr=img_file, size=(120, 160, 3))
                         img_arr = np.asarray(img_file)
                         X.append(img_arr)
                         y.append(label)
    X = np.asarray(X)
    y = np.asarray(y)
    return X,y

In [177]:
# load the data sets, training and testing

X, y = get_data('C:/Users/NeverGiveUp/Desktop/Update Input File/')

In [178]:
# split the data randomnly
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.20, random_state = 0)

In [179]:
# import transformer class
from sklearn.preprocessing import StandardScaler

In [180]:
# convert data from 3d to 2d array
nsamples, nx, ny = X_train.shape
nsamples2, nx2, ny2 = X_test.shape
X_train = X_train.reshape((nsamples,nx*ny))
X_test = X_test.reshape((nsamples2,nx2*ny2))

In [181]:
# instantiate the Logist. Reg classifier
# transform X train and X test data
log_reg = LogisticRegression(C = 1.0, random_state = 0)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [182]:
# convert y train and y test data to fit the classifier
nsamples  = y_train.shape
nsamples2 = y_test.shape
y_train = y_train.reshape(nsamples)
y_test = y_test.reshape(nsamples2)

In [183]:
# fit the logReg classifier with training data
log_reg.fit(X_train, y_train)

C:\Users\NeverGiveUp\Anaconda3\envs\MyEnv\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [184]:
# perfrom prediction on X test data
y_pred = log_reg.predict(X_test)

In [185]:
print(y_pred)

['0' '0' '1' '0' '1' '0' '0' '0' '1' '0' '1' '0' '0' '0' '1' '0' '0' '0'
 '1' '1' '1' '0' '0' '0' '1' '1' '0' '1' '0' '0' '0' '1' '1' '1' '0' '1'
 '0' '0' '1' '1' '0' '0' '1' '0' '1' '0' '1' '0' '0' '1' '0' '0' '0' '0'
 '0' '0' '1' '0' '0' '1' '0' '1' '1' '0' '0' '0' '1' '0' '1' '1' '0' '0'
 '0' '0' '0' '1' '0' '0' '0' '1' '1' '1' '1' '0' '0' '1' '0' '0' '0' '0']


In [186]:
# get the confusion accuracy by comparing the prediction result to the test labels, y_test
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [187]:
print(cm)

[[37  9]
 [19 25]]


In [188]:
accuracy = accuracy_score(y_test, y_pred)

In [189]:
print(accuracy)

0.6888888888888889


In [190]:
# Elect  parameters for the grid_search to find the best params and the best model.
# parameters here are just estimation
parameters = [{'C':[1, 10, 100], 'class_weight': [None], 'dual': [False], 'fit_intercept': [True],
                   'intercept_scaling': [1], 'l1_ratio': [None], 'max_iter': [100],
                   'multi_class': ['auto'] , 'n_jobs': [None], 'penalty' : ['l2']}]


In [191]:
grid_search = GridSearchCV(estimator = log_reg, param_grid = parameters,
                          scoring = 'accuracy', cv = 10, n_jobs = -1)

In [192]:
grid_search = grid_search.fit(X_train, y_train)

C:\Users\NeverGiveUp\Anaconda3\envs\MyEnv\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [193]:
accuracy = grid_search.best_score_

In [194]:
print(accuracy)

0.7194444444444444


In [195]:
print(grid_search.best_params_)

{'C': 10, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2'}


In [196]:
log_reg = LogisticRegression(C = 10)

In [197]:
log_reg.fit(X_train, y_train)

C:\Users\NeverGiveUp\Anaconda3\envs\MyEnv\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [198]:
y_pred = log_reg.predict(X_test)

In [199]:
cm = confusion_matrix(y_test, y_pred)

In [200]:
print(cm)

[[38  8]
 [18 26]]


In [201]:
accuracy

0.7194444444444444